In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import selenium
import time
from bs4 import BeautifulSoup
import pandas as pd
import os
from urllib.request import urlretrieve


In [14]:
ITEMS_COUNT = 100

In [15]:
def start_crawling(driver):
    s = Service("D:\chromedriver.exe")
    
    driver.maximize_window()

    # 접속하기
    url = 'https://www.musinsa.com/app/?utm_source=google_shopping&utm_medium=sh&utm_campaign=pmax_ongoing&source=GOSHSAP001&utm_source=google_shopping&utm_medium=sh&utm_campaign=pmax_ongoing&source=GOSHSAP001&gad_source=1&gclid=Cj0KCQjw2uiwBhCXARIsACMvIU1UMS64MMDr5AnHmY9HCoA5rr0_lsqU7qfdTBcMfRIqPs1go8c8IJIaAhmFEALw_wcB'

    # 지정한 웹 페이지 열기
    driver.get(url)
    time.sleep(1)


In [16]:
def access_middle_category(link, driver):
    driver.get(link['href'])

In [17]:
def set_to_1day_sales_ranking(driver):

    # 판매순 토글 클릭
    driver.find_element('xpath', '//*[@id="goods_list"]/div[2]/div[1]/a[7]/span/i').click()

    # 1일로 설정
    driver.find_element('xpath', '//*[@id="layerSorting_sale"]/div/label[1]').click()

In [18]:
def access_product_link(product_link, driver):
    driver.get(product_link['href'].replace('//', 'https://'))
    time.sleep(1)
    
    # 품번만 받아서 전달
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    product_num = soup.select('span.product-detail__sc-achptn-4.cobjEx')[2].text

    return product_num


In [19]:
def get_product_information_excluding_review(driver):

    # 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(1)

    # 브랜드
    brand = soup.select('div.product-detail__sc-achptn-6.TehCn > a')[0]['title']

    # 품번
    product_num = soup.select('span.product-detail__sc-achptn-4.cobjEx')[2].text

    # 누적판매(1년)
    cumulative_sales = soup.select('span.product-detail__sc-achptn-4.cobjEx')[8].text

    # 좋아요
    heart = soup.select('span.product-detail__sc-achptn-4.coaOzR')[0].text

    # 정가(무신사 판매가)
    regular_price = soup.select('span.product-detail__sc-1p1ulhg-7.JZAP')[0].text

    # 할인가(무신사 회원가)
    try:
        discount_sales = soup.select('span.product-detail__sc-1p1ulhg-7.kijFAA')[0].text.split('~')[0].strip()  
    except:
        discount_sales = soup.select('span.product-detail__sc-1p1ulhg-7.jBxBWz')[0].text

    # 이미지 링크
    img_thumbnail = soup.select('li.product-detail__sc-p62agb-8.iqAHsp')[0]
    img_links = soup.select('ul.product-detail__sc-p62agb-7.deRVDj > li.product-detail__sc-p62agb-8.hFLxgF > img')
    img_links += img_thumbnail

    if not os.path.exists(f"../images/{product_num}"):
            os.mkdir(f"../images/{product_num}")

    img_urls = []
    for i, img_link in enumerate(img_links):
        img_link = img_link['src']
        img_urls.append(img_link)

        dir_path = f'../images/{product_num}/{product_num}_{i}.png'

        # 파일이 존재하지 않을 때만 다운로드
        if not os.path.exists(dir_path):
            urlretrieve(img_link, dir_path)

    # 나이
    ''' 
    0 : ~18세
    1 : 19~23세
    2 : 24~28세
    3 : 29~33세
    4 : 34~39세
    5 : 40세~
    '''
    ages = soup.select('span.product-detail__sc-riuadz-12.cFgaXH')
    
    # 남성 구매 비율
    man = soup.select('span.product-detail__sc-18glxph-9.YnCTG')[0].text

    # 여성 구매 비율
    woman = soup.select('span.product-detail__sc-18glxph-9.YnCTG')[1].text

    # 데이터를 담을 딕셔너리 생성
    data = {
        'brand': brand,
        'product_num': product_num,
        'cumulative_sales_1yr': cumulative_sales,
        'heart': heart,
        'regular_price': regular_price,
        'discount_sales': discount_sales,
        'ages' : ages,
        'img_link' : img_urls,
        'man': man,
        'woman': woman,
        
    }

    return data
    

In [20]:
def crawling_each_types(driver):
    # 스타일 리뷰부터 크롤링
    df_style_reviews = crawling_reviews(driver)
    
    # 상품 후기 클릭
    driver.find_element('xpath', '//*[@id="estimateBox"]/div[2]/ul/li[2]').click()

    # 상품 후기 크롤링
    df_product_reviews = crawling_reviews(driver)

    # 일반 후기 클릭
    driver.find_element('xpath', '//*[@id="estimateBox"]/div[2]/ul/li[3]').click()

    # 일반 후기 크롤링
    df_normal_reviews = crawling_reviews(driver)

    return df_style_reviews, df_product_reviews, df_normal_reviews



def crawling_reviews(driver):

    time.sleep(3)
    # 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(1)

    # 화면 살짝 오른쪽으로
    driver.execute_script("window.scrollBy(200, 0)")  

    # 현재 페이지 리뷰 수집
    reviews = soup.select('div.review-list-wrap > div.review-list')

    # 리뷰 리스트
    review_list = []

    # 마지막 페이지
    last_page = soup.select('div.box_page_msg')[0].text.split()[0]
    
    while True:
        # 버튼 눌러줄때마다 최신화 해줘야 함
        next_buttons = get_next_buttons(driver)

        # 한 페이지의 페이지 버튼 개수(끝 페이지 처리 위한 변수)
        len_buttons = len(next_buttons)

        # ex) 첫 for문에서는 1,2,3,4,5 페이지의 리뷰만 수집함.
        for i in range(len_buttons):
            for review in reviews:
                # review_list 에 리뷰 내용 추가
                review_list.append(get_one_review(review))
            
            next_buttons[i].click()
            time.sleep(1)

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            time.sleep(1)

            reviews = soup.select('div.review-list-wrap > div.review-list')

            next_buttons = get_next_buttons(driver)

        # 각 5의 배수 페이지 리뷰 가져오기
        for review in reviews:
                # review_list 에 리뷰 내용 추가
                review_list.append(get_one_review(review))


        # 5n 페이지에서 5n+1 페이지로 넘어갈 때
        if len_buttons == 4:
            if next_buttons[i].text == last_page:
                break
            
            driver.find_elements('css selector', 'a.fa.fa-angle-right.paging-btn.btn.next')[0].click()
            time.sleep(1)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            time.sleep(1)
            reviews = soup.select('div.review-list-wrap > div.review-list')
            
        # 마지막 페이지일 때 break
        else:
            break


    return pd.DataFrame(review_list)

    

def get_next_buttons(driver):
    next_buttons = driver.find_elements('css selector', 'a.paging-btn.btn')
    next_buttons = next_buttons[3:-2]

    return next_buttons



def get_one_review(review):

    # 작성자 이름
    name = review.select('div.review-profile > div.review-profile__text-wrap > div > p.review-profile__name')[0].text
    
    # 별 갯수(퍼센트)
    star = review.select('span.review-list__rating__active')[0]['style']

    # 도움돼요 수
    helpful = review.select('label.review-evaluation-button--type3__item.review-evaluation-button--type3__item--button01 > span.review-evaluation-button--type3__count')[0].text

    # 스타일좋아요 수. 스타일 후기에서만 존재
    try:
        style_good = review.select('label.review-evaluation-button--type3__item.review-evaluation-button--type3__item--button02 > span.review-evaluation-button--type3__count')[0].text
    except:
        style_good = ''

    # 댓글 내용
    content = review.select('div.review-contents__text')[0].text

    # 메타 데이터
    meta = review.select('ul.review-evaluation--type3__list > li.review-evaluation--type3__item')

    data = {
    'name': name,
    'star': star,
    'helpful': helpful,
    'style_good': style_good,
    'content': content,
    'meta': meta
    }

    return data




In [21]:
def main():
    s = Service("D:\chromedriver.exe")
    driver = webdriver.Chrome(service=s)
    
    # 무신사 접속
    start_crawling(driver)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(1)

    # 대분류 카테고리 받아오기
    main_categories = soup.select('div.sc-nw870h-1.knbIYx > section > div > nav > div[role]' )

    # 중분류 링크들 받아오기
    middle_categories_links = []
    for i in range(1, 6):
        middle_categories_links += main_categories[i].select('li > a[href]')[1:]
    
    # # test용. 두 개의 중분류만
    # middle_categories_links = middle_categories_links[:2]

    # 중분류 하나씩 탐방
    for i,  middle_categories_link in enumerate(middle_categories_links):

        # 해당 중분류 페이지로 이동
        access_middle_category(middle_categories_link, driver)
        time.sleep(1)

        # 판매순 1일로 설정
        set_to_1day_sales_ranking(driver)

        # 파싱
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(1)
        
        # 상품 링크 수집. 일단은 한페이지에서만. 100개 수집하려면 수정 필요
        product_links = soup.select('div.list-box.box > ul > li.li_box > div.li_inner > div.article_info > p.list_info > a[href]')

        # test용. 각 중분류에서 두개씩만
        product_links = product_links[:2]

        # 각 상품을 한 행으로 저장 (리뷰 제외)
        products_list = []
        reviews_list = []

        # 개별 상품의 정보 추출
        for product_link in product_links:

            # 상품 링크 접속, 품번 가져오기(key값으로 쓸 용도)
            product_num = access_product_link(product_link, driver)
            time.sleep(1)

            if not os.path.exists(f"../images"):
                os.mkdir(f"../images")

            if not os.path.exists(f"../dataframes/reviews"):
                os.mkdir(f"../dataframes/reviews")

            if not os.path.exists(f"../dataframes/reviews/{product_num}"):
                os.mkdir(f"../dataframes/reviews/{product_num}")

            # 리뷰 제외한 정보 추출
            products_list.append(get_product_information_excluding_review(driver))

            # 리뷰 정보 추출
            df_style_reviews, df_product_reviews, df_normal_reviews = crawling_each_types(driver)

            # csv로 저장
            df_style_reviews.to_csv(f'../dataframes/reviews/{product_num}/style_reviews.csv', index=False)
            df_product_reviews.to_csv(f'../dataframes/reviews/{product_num}/product_reviews.csv', index=False)
            df_normal_reviews.to_csv(f'../dataframes/reviews/{product_num}/normal_reviews.csv', index=False)

        df_products = pd.DataFrame(products_list)
        df_products.to_csv(f'../dataframes/middle_categories/middle_category_{i}.csv', index=False)



    

In [22]:
if __name__ == '__main__':
    main()

ElementClickInterceptedException: Message: element click intercepted: Element <a href="javascript:void(0);" class="paging-btn btn" onclick="ReviewPage.goPage(...); return false;">122</a> is not clickable at point (1027, 74). Other element would receive the click: <div class="review-contents__text">...</div>
  (Session info: chrome=123.0.6312.123)
Stacktrace:
	GetHandleVerifier [0x00007FF753F17032+63090]
	(No symbol) [0x00007FF753E82C82]
	(No symbol) [0x00007FF753D1EC65]
	(No symbol) [0x00007FF753D6BB34]
	(No symbol) [0x00007FF753D69954]
	(No symbol) [0x00007FF753D67164]
	(No symbol) [0x00007FF753D65EF9]
	(No symbol) [0x00007FF753D5A708]
	(No symbol) [0x00007FF753D86FDA]
	(No symbol) [0x00007FF753D5A00A]
	(No symbol) [0x00007FF753D871F0]
	(No symbol) [0x00007FF753DA3412]
	(No symbol) [0x00007FF753D86D83]
	(No symbol) [0x00007FF753D583A8]
	(No symbol) [0x00007FF753D59441]
	GetHandleVerifier [0x00007FF7543125AD+4238317]
	GetHandleVerifier [0x00007FF75434F70D+4488525]
	GetHandleVerifier [0

In [24]:
pd.read_csv("D:\\kwangwoon\\2024_1\\산학\dataframes\\reviews\\TRSKN19BK\\normal_reviews.csv")

,name,star,helpful,style_good,content,meta
0,LV.3 모거니야,width: 20%,5,NaN,여기서 2상품 주문 했는대 하나제품은 오고 하나제품은 오지않고 상품2개는 배송완료라...,"[<li class=""review-evaluation--type3__item"">사이..."
1,LV.5 daumdjml,width: 100%,3,NaN,완전 니트로된 가디건이라서 부드러워서 좋긴한데얼마나 입을 수 있을지 .. 잘관리해...,"[<li class=""review-evaluation--type3__item"">사이..."
2,LV.6 네버다이다이,width: 100%,2,NaN,생각했던것보다 얇아서 고민입니다...두께감이 좀 있을줄 알았는데...,"[<li class=""review-evaluation--type3__item"">사이..."
3,LV.4 꽭현세,width: 100%,1,NaN,"옷감도 좋고 가격대비 스타일링하기 좋아요, 스트릿패션으로 딱 굳","[<li class=""review-evaluation--type3__item"">사이..."
4,LV.5 뉴비_825d20cc,width: 100%,1,NaN,옷 이쁜데... 팔 부분에 올이 나간 상태로 왔어요... 그거만 빼면 좋아요ㅜㅜ,"[<li class=""review-evaluation--type3__item"">사이..."
...,...,...,...,...,...,...
189,LV.3 닉네임을 정하시오,width: 100%,0,NaN,사이즈 일부러 큰거 샀는데 만족합니다 디자인도 예뻐요,"[<li class=""review-evaluation--type3__item"">사이..."
190,LV.4 외모원탑,width: 100%,0,NaN,가을이나 봄에 입고 다니기 좋은 두께고 디자인도 이쁘네요,"[<li class=""review-evaluation--type3__item"">사이..."
191,LV.5 뉴비_dd906db1,width: 100%,0,NaN,편해보이고 색도 이뻐서 날 풀리면 자주 입을거 같아요,"[<li class=""review-evaluation--type3__item"">사이..."
192,LV.3 야물딱지고,width: 100%,0,NaN,디자인도 괜찮고 색감도 굳굳 사이즈도 적당해서 입기 편해요,"[<li class=""review-evaluation--type3__item"">사이..."
